In [1]:
%matplotlib inline

In [9]:
import matplotlib.pyplot as plt
import numpy as np
import random
from tqdm import trange, tqdm

In [3]:
from rsnn.rsnn.generator import NetworkGenerator
from rsnn.rsnn.network import Network
from rsnn.rsnn.neuron import Neuron, Synapse
from rsnn.spike_train.generator import PeriodicSpikeTrainGenerator
from rsnn.spike_train.periodic_spike_train import MultiChannelPeriodicSpikeTrain, PeriodicSpikeTrain

/Users/haguettaz/Documents/RSNN/src/rsnn/rsnn/network.py:8: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [4]:
plt.style.use('paper')

# Create Neural Network

In [5]:
num_neurons = 2000
nominal_threshold = 1.0
absolute_refractory = 5.0
relative_refractory = 5.0

num_synapses = 500
synapse_beta = 5.0
synapse_delay_lim = (1.0, 60.0)

period = 100.0
firing_rate = 0.1

In [6]:
eps, gap, slope = 1.0, 1.0, 0.5
weights_lim = (-0.1, 0.1)

In [7]:
network = Network(num_neurons, nominal_threshold, absolute_refractory, relative_refractory)

In [ ]:
spike_train_generator = PeriodicSpikeTrainGenerator(firing_rate, absolute_refractory, relative_refractory)

In [ ]:
for period in tqdm([100, 200, 400]):
    spike_trains = spike_train_generator.rand(period, num_neurons)  
    for n_ in network.neurons:
        n_.spike_train = spike_trains.spike_trains[n_.idx]
        
    for i in trange(20):
        for num_unique_neurons in [1, num_synapses]:
            neuron = random.choice(network.neurons)
            neuron_pool = random.sample(network.neurons, num_unique_neurons)
            neuron.synapses =[Synapse(
                idx,
                neuron_pool[idx % num_unique_neurons],
                synapse_beta,
                random.uniform(*synapse_delay_lim))
                for idx in range(num_synapses)]
            neuron.init_template_single(eps, gap, slope)
            neuron.memorize(weights_lim, "l1")
            network.save_to_file(f"networks/network_sparse_{period}_{num_unique_neurons}_{i}.pkl")
            neuron.synapses = []